# Assignment 2

# 1. Regression

In this assignment, you will explore the California Housing Prices dataset. Your task is to apply various regression techniques, specifically Kernel Ridge Regression (KRR), Bayesian Linear Regression, and Gaussian Process Regression, to predict housing prices.


### Dataset
We will use the "California Housing Prices" dataset from the `sklearn.datasets` module for ease of access. 

### Objectives
1. Perform exploratory data analysis (EDA) to understand the dataset.
2. Apply Kernel Ridge Regression (KRR) with different hyperparameters and analyze the results.
3. Implement Bayesian Linear Regression and discuss the posterior distributions.
4. Explore Gaussian Process Regression and visualize the prediction uncertainties.
5. Compare the performance of the three regression techniques and discuss your findings.



## Tasks

### 1. Data Exploration and Preprocessing
- Use the knowledge you gain from previouse assignment on this dataset. 
- You do not have to repeat the visualization of the data, only remeber the important fact that you gain by analysing it. 

#### 2. Kernel Ridge Regression (KRR)
- Apply KRR to the dataset.
- Experiment with different kernels and regularization parameters.
- Analyze the performance and discuss how different hyperparameters impact the model.

#### 3. Bayesian Linear Regression
- Implement Bayesian Linear Regression. You can use  Bayesian Ridge Regression function from Scikit-Learn.
- Visualize the posterior distributions of the coefficients (at least one coefficient).
- Discuss the insights gained from the posterior analysis.

#### 4. Gaussian Process Regression
- Apply Gaussian Process Regression to the dataset.
- Visualize the prediction uncertainties.
- Discuss how the Gaussian Process handles uncertainty in predictions.

#### 5. Comparative Analysis
- Compare the results obtained from KRR, Bayesian Linear Regression, and Gaussian Process Regression.
    - Visualize the regression fit provided by each model and compare.
    - Use at least one of the metrics: MSE, RMSE, R2
- Evaluate and discuss the performance, computational efficiency, and ease of interpretation of each model.
    - For Bayesian Linear Regression, analyze the posterior distributions of the coefficients. For KRR, discuss the interpretability of the kernel.



# 2. Outlier detection
In this assignment, you can choose one of the below dataset and your task is to identify the outliers in them using various machine learning techniques.


## Dataset

You can choose One of the below datasets:
1. The dataset is a small sample from the Fashion MNIST dataset with manually added outliers. The data will be provided in the form of two numpy arrays: `images` and `labels`. (Note:check for missing data)
2. You will generate your data with outliers:
    * Use NumPy to create data points that follow a normal distribution. This forms the "normal" part of your dataset. 
    * Manually add data points that are significantly different from the normal data. 
    * These points should be distant from the mean of the normal data to be considered outliers.
    * Combine the normal data points and outliers into a single dataset.

### Objectives
1. Perform exploratory data analysis (EDA) to understand the dataset.
2. Implement PCA (Principal Component Analysis) for dimensionality reduction and visualize the results.
3. Use K-means clustering to identify potential outliers.
4. Apply t-SNE (t-Distributed Stochastic Neighbor Embedding) for visualization and detect anomalies.
5. (Optional) Design and train an autoencoder and use reconstruction error to find outliers.
6. Compare the effectiveness of the above methods in outlier detection.





## Tasks

#### 1. Exploratory Data Analysis (EDA)
- Load the dataset and visualize some images.
- Plot the distribution of the different classes in the dataset.

#### 2. PCA for Dimensionality Reduction
- Implement PCA to reduce the dimensionality of the dataset.
- Visualize the data in the reduced dimension space.

#### 3. Choose one of the below tasks:
* QDA for Outlier Detection
    - Apply QDA  to the dataset.
    - Analyze how the QDA  decision boundary help in outlier detection.

* K-means Clustering
    - Apply K-means clustering on the dataset.
    - Identify clusters that potentially contain outliers.

#### 5. t-SNE for Visualization
- Apply t-SNE to the dataset and visualize the results.
- Discuss how t-SNE helps in identifying outliers.


#### 6. Autoencoder for Outlier Detection (Optional)
- Design and train an autoencoder on the dataset.
- Use the reconstruction error to identify images that are outliers.


#### 7. Comparative Analysis
- Compare the results of the models you chose to study.
- Discuss the effectiveness and limitations of each method in outlier detection.

## How to Submit
- First, a Jupyter Notebook containing all the code, comments, and analysis.
- Second report cells in the same Jupyter Notebook, summarizing your findings, including results and a discussion of the results.
- Finally convert the Jupyter Notebook to PDF.
- **Don't write your name**.
- Upload the PDF into convas.

## Evaluation Criteria (peer grading)
- Correctness of the implementation of the models. (2 points)
- Quality of the EDA and preprocessing steps. (1 point)
- Depth of the analysis in comparing the models.(1 point)
- Clarity and organization of the submitted report and Jupyter Notebook. (1 point)

